# [과제 2] 회귀분석
### - Ch 1, Ch 2를 토대로 자유롭게 회귀분석과 회귀진단을 진행해주세요.
### - 주석으로 설명 및 근거 자세하게 달아주시면 감사하겠습니다. :)

In [324]:
%matplotlib inline 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

warnings.filterwarnings(action='ignore')

# Data 

데이터 출처 : https://www.kaggle.com/avikasliwal/used-cars-price-prediction 

< y > 
* **Price** : The price of the used car in INR Lakhs.



< X > 
* Name : The brand and model of the car
* Location : The location in which the car is being sold or is available for purchase.
* Year : The year or edition of the model.
* Kilometers_Driven : The total kilometres driven in the car by the previous owner(s) in KM.
* Fuel_Type : The type of fuel used by the car. (Petrol, Diesel, Electric, CNG, LPG)
* Transmission : The type of transmission used by the car. (Automatic / Manual)
* Owner_Type : Whether the ownership is Firsthand, Second hand or other.
* Mileage : The standard mileage offered by the car company in kmpl or km/kg
* Engine : The displacement volume of the engine in CC.
* Power : The maximum power of the engine in bhp.
* Seats : The number of seats in the car.
* New_Price : The price of a new car of the same model.

In [325]:
# Load Data 
data = pd.read_csv("assignment2_data.csv")
data.head(10)

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
5,5,Hyundai EON LPG Era Plus Option,Hyderabad,2012,75000,LPG,Manual,First,21.1 km/kg,814 CC,55.2 bhp,5.0,NaN,2.35
6,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50
7,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8.0,21 Lakh,17.50
8,8,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,Diesel,Manual,First,20.54 kmpl,1598 CC,103.6 bhp,5.0,NaN,5.20
9,9,Tata Indica Vista Quadrajet LS,Chennai,2012,65932,Diesel,Manual,Second,22.3 kmpl,1248 CC,74 bhp,5.0,NaN,1.95


In [326]:
#null value 확인
data.isnull().sum()

Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  42
New_Price            5195
Price                   0
dtype: int64

In [327]:
# 사용할 변수와 target 설정
# numerical column만을 사용하여 price 예측
data_variabe = ['Kilometers_Driven','Mileage','Engine','Power','Seats','Price']
new_dataframe=data[data_variabe]
new_dataframe.shape

(6019, 6)

In [328]:
#preprocessing(결측치제거, 단위값제거)
new_dataframe.dropna(inplace=True)
new_dataframe.drop(new_dataframe[new_dataframe['Power']=='null bhp'].index, inplace=True)

In [329]:
new_dataframe['Mileage']=new_dataframe['Mileage'].str.replace(' kmpl','').str.replace(' km/kg','').astype(float)
new_dataframe['Engine']=new_dataframe['Engine'].str.replace(' CC','').astype(float)
new_dataframe['Power']=new_dataframe['Power'].str.replace(' bhp','').astype(float)

In [330]:
# variable, target 분리
x = new_dataframe.drop(columns='Price')
y = new_dataframe['Price']
x

,Kilometers_Driven,Mileage,Engine,Power,Seats
0,72000,26.60,998.0,58.16,5.0
1,41000,19.67,1582.0,126.20,5.0
2,46000,18.20,1199.0,88.70,5.0
3,87000,20.77,1248.0,88.76,7.0
4,40670,15.20,1968.0,140.80,5.0
...,...,...,...,...,...
6014,27365,28.40,1248.0,74.00,5.0
6015,100000,24.40,1120.0,71.00,5.0
6016,55000,14.00,2498.0,112.00,8.0
6017,46000,18.90,998.0,67.10,5.0


In [331]:
#train, test split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,shuffle=True ,random_state=42)

In [332]:
# scaler 
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [333]:
#train 
model = LinearRegression() 
model.fit(x_train_scaled,y_train)

LinearRegression()

In [334]:
#test
pred = model.predict(x_test_scaled)

In [335]:
#metric
mse = mean_squared_error(y_test, pred)
rmse = mse**0.5
rmse

6.904062608685381